This workbook converts the schoolList.json file from the web scraping process into a prisma seed file. The prisma seed file is used to populate the database with the school data.

This conversion is partial manual since the school from the SFUSD website are not a perfect match for the data submitted to the CA Department of Education. The workbook attempts to match the records on a combination of zipcode and school name, but the algorithm is not perfect. In cases of low quality matches, the casid (short for CA School ID) can be confirmed by looking at the SARC document for the school (located in Google Drive folders).

This workbook also removes the schools that are not in the California Department of Education databases. These schools are typically preschools or non-degree granting institutions.

In [5]:
# this function finds the best match for a school name in a given zipcode
from difflib import SequenceMatcher as SM

def best_match(qmap, school_name, zipcode):
    if zipcode not in qmap:
        return None

    best_ratio = 0
    best_match = None

    for school in qmap[zipcode]:
        ratio = SM(None, school_name, school["school_name"]).ratio()
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = school
    return {"best_match": best_match, "ratio": best_ratio}


In [6]:
# this function generates a hash for a given string
# this is more realible in this workflow than using
# the string itself since Excel is used in the
# review/correction process and it butchers the
# Chinese characters etc.

import hashlib

def generate_hash(input_string: str) -> str:
    hash_object = hashlib.sha256()
    hash_object.update(input_string.encode('utf-8'))    
    return hash_object.hexdigest()

In [7]:
# this secion reads the school list from the CA Dept of Education database
# and creates a map of schools by zipcode which is used in the fn best_match
# to find the best match for a given school name in a given zipcode
# the map is then used to match the schools in the schoolList.json file
# to the schools in the CA Dept of Education database by adding the school_code (casid)
# to the school record.

# It also creates a CSV file with the matches for review and correction. Excel can be used
# to correct low quality/erroneous matches to produce an 'actions.csv' file that informs
# the next step in the workflow on what changes need to be made to the schoolList.json file.

from dotenv import load_dotenv
import psycopg2
import os
import json
import csv

load_dotenv()

db_password = os.getenv("DB_PASSWORD")
db_user = os.getenv("DB_USER")
db_name = os.getenv("DB_NAME")

connection = psycopg2.connect(
    dbname=db_name, user=db_user, password=db_password, host="localhost", port="5432"
)

cursor = connection.cursor()

# SFUSD = '68478'
cursor.execute(
    "SELECT school_code, school_name, zip_code from entities where county_code = '38';"
)

results = cursor.fetchall()

qmap = {}
for scode, sname, szip in results:
    if szip in qmap:
        qmap[szip].append({"school_code": scode, "school_name": sname})
    else:
        qmap[szip] = [{"school_code": scode, "school_name": sname}]

cursor.close()
connection.close()


#### read the json file
with open("schoolList.json", "r", encoding="utf-8") as file:
    data = json.load(file)

csv_output = [
    [
        "school_hash",
        "school_uid",
        "schoolLabel",
        "schoolCode",
        "zip",
        "match_school_id",
        "match_school_name",
        "match_ratio",
    ]
]

for school in data:
    zip = f"no zip for {school['schoolLabel']}"
    if "geolocations" in school:
        location = school["geolocations"][0]["addressDetails"]
        schoolCode = school["schoolCode"] if "schoolCode" in school else "<missing>"
        if "PostalCode" in location:
            zip = location["PostalCode"].split("-")[0]
            
            best = best_match(qmap, school["schoolLabel"], zip)
            match_school_id = best["best_match"]["school_code"] if best else ""
            match_school_name = best["best_match"]["school_name"] if best else ""
            match_ratio = best["ratio"] if best else 0.0

            school_unique_id = f"{zip}-{school['schoolLabel']}"
            school_hash = generate_hash(f"{school_unique_id}")
            
            csv_output.append(
                [
                    str(school_hash),
                    school_unique_id,
                    school["schoolLabel"],
                    schoolCode,
                    zip,
                    match_school_id,
                    match_school_name,
                    f"{float(match_ratio):.2f}",
                ]
            )
            school["school_hash"] = school_hash
            school["school_unique_id"] = school_unique_id
            school["casid"] = match_school_id
            # print(f"added {school_unique_id} ({school_hash})")
        else:
            print(f"no zip for {school['schoolLabel']}")
    else:
        print(f"no geolocations for {school['schoolLabel']}")

# Write the CSV
with open("matches.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    for row in csv_output:
        writer.writerow(row)

### Fix the bad school_codes

Unfortunately, this is a manual part of the workflow. The school zipcodes are not always accurate which leads to bad matches. Moreover, the matching algorithm is not perfect. Where the match looks incorrect, the `casid` can be confirmed by looking at the SARC document for the school (located in Google Drive folders).

The suggesed workflow is to open the `matches.csv` produced above in either Excel or another speadsheet package to review the matches. Be careful not to let Excel remove the zero-padding from the identifiers.

After reviewing the matches, you need to create another CSV file with the correct casids. The file should have the following columns with no header row:

- `school_hash`: The hash of the school name and zipcode (from the `matches.csv` file)
- `casid`: The correct casid for the school
- `instruction`: A note about the change either `KEEP`, `DELETE`, or `UPDATE`

For example:

```csv
797c2568e4c71628d72d86379d7282ccc8a65a5aa4f13ea28958ae619f1736e8,6089569,OVERWRITE
98f4714e5aade9a9465c99ccb07582aa0309787319074631df4b81f73a499242,6040752,DELETE
e372d38bbc0d25217f22c4568fe6688df8f6ce9a296799d196b727469846a47b,6040752,KEEP
19c4204704adb9868eb23780ee585451e596feb8346812a18382a2a108a991be,0102103,DELETE
f948b5c086e43e7f885e93a8bd9d3cb8bb7f50388fd0751ab42e10ee76febfb0,3831765,KEEP
e5ad922242f5110e8656a050a9affc0e595fcc42cbbbc6f7a5bc2bb7f6f30edb,6102479,KEEP
```

This file should be saved as `actions.csv` in the same directory as this notebook. It will be used in the next step to update the `schoolList.json` file.

In [ ]:
# This section reads the actions.csv file and applies the changes to the schoolList.json file
# by removing the schools that have been marked for deletion and updating the casid to the schools
# the instructions in the actions.csv file. The updated schoolList.json file is then written to
# schoolList_hashed.json

with open("actions.csv", mode="r", newline="", encoding="utf-8") as file:
    reader = csv.reader(file)
    actions = [row for row in reader]

del_list = [action[0] for action in actions if action[2] == "DELETE"]
filtered = [school for school in data if school["school_hash"] not in del_list]

# create a dictionary of school_hash to casid
casid_lookup = {action[0]: action[1] for action in actions if action[2] != "DELETE"}

for school in filtered:
    if not school["school_hash"] in casid_lookup:
        print(f"no action for {school['schoolLabel']}")
    else:
        school["casid"] = casid_lookup.get(school["school_hash"])

# Write the JSON
with open("schoolList_hashed.json", "w", encoding="utf-8") as file:
    json.dump(data, file, indent=2)